In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from langgraph.graph import END, StateGraph
from typing_extensions import TypedDict
from typing import List
from pprint import pprint

# Load environment variables
load_dotenv()

# Define constants
LOCAL_LLM = "phi3:3.8b"
PDF_PATH = 'Basic-Fantasy-RPG-Rules-r139.pdf'
CHUNK_SIZE = 500
CHUNK_OVERLAP = 150
VECTOR_COLLECTION_NAME = "rag-chroma"

# Load and process documents
loader = PyMuPDFLoader(PDF_PATH)
bf_data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
bf_split_data = text_splitter.split_documents(bf_data)

# Initialize vectorstore
vectorstore = Chroma.from_documents(
    documents=bf_split_data,
    collection_name=VECTOR_COLLECTION_NAME,
    embedding=GPT4AllEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [13]:
# Define retrieval grader + generation prompt + hallucination grader + answer grader
llm = ChatOllama(model=LOCAL_LLM, format="json", temperature=0)
retrieval_grader_prompt = PromptTemplate(
    template="""system You are a grader assessing relevance of a retrieved document to user prompts. 
    If the document contains context related to the user prompt, grade it as relevant. 
    It does not need to be a stringent test. 
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. 
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
    user Here is the retrieved document: \n\n {document} \n\n Here is the user question: {question} \n assistant""",
    input_variables=["question", "document"],
)
retrieval_grader = retrieval_grader_prompt | llm | JsonOutputParser()

generation_prompt = PromptTemplate(
    template="""system Imagine that you are a wise and ancient wizard. You are an expert assistant for a Dungeon Master 
    during a tabletop role-playing session. Your goal is to respond to prompts directly with detailed and accurate 
    information derived from retrieved context. Use only your retrieved context to respond to the prompt. 
    Limit your response to no more than 10 lines. Do not open your response with the phrase "As a". 
    user Question: {question} Context: {context} Answer: assistant""",
    input_variables=["question", "context"],
)
rag_chain = generation_prompt | ChatOllama(model=LOCAL_LLM, temperature=0) | StrOutputParser()

hallucination_grader_prompt = PromptTemplate(
    template="""system You are a grader assessing whether a response is grounded in / supported by a set of facts that are 
    present in retrieved context. Give a score of 'yes' or 'no' to indicate whether the answer is grounded in / supported 
    by a set of facts that are present in retrieved context. Provide the score as a JSON with a single key 'score' and no 
    preamble or explanation. user Here are the facts: \n ------- \n {documents} \n ------- \n Here is the answer: 
    {generation} assistant""",
    input_variables=["generation", "documents"],
)
hallucination_grader = hallucination_grader_prompt | llm | JsonOutputParser()

answer_grader_prompt = PromptTemplate(
    template="""system You are a grader assessing whether an answer is useful to resolve a question. Give a score of 'yes' 
    or 'no' to indicate whether the answer is useful to resolve a question. Provide the score as a JSON with a single key 
    'score' and no preamble or explanation. user Here is the answer: \n ------- \n {generation} \n ------- \n Here is the 
    question: {question} assistant""",
    input_variables=["generation", "question"],
)
answer_grader = answer_grader_prompt | llm | JsonOutputParser()

In [94]:
### State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

### Nodes

def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score["score"]
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


### Conditional edge

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    filtered_documents = state["documents"]
        # We have relevant documents, so generate answer
    print("---DECISION: GENERATE---")
    return "generate"


### Conditional edge


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score["score"]

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score["score"]
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS---")
        return "not supported"


from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae

# Build graph
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "generate": "generate",
    },
)
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)

In [95]:
# Compile and test the workflow
app = workflow.compile()
inputs = {"question": "What are the stats of a Goblin?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
        pprint(value)

ValueError: At 'generate' node, 'grade_generation_v_documents_and_question' branch found unknown target 'websearch'